<a href="https://colab.research.google.com/github/khawla-T/NeuralNetwork/blob/main/sudidialect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/iwan-rg/Saudi-Dialect-Irony-Dataset/tree/main


In [1]:
pip install pyspellchecker

In [2]:
pip install arabert

In [3]:
pip install SpellChecker

In [4]:
pip install spellchecker

In [5]:
pip install transformers

In [6]:
! pip install -U accelerate
! pip install -U transformers

In [7]:
#from spellchecker import SpellChecker
from tqdm import tqdm
import re
import pyarabic.araby as araby
import pandas as pd

Preprocessing

Before training the model, the data is preprocessed by performing the following steps:

drop all word or letters, which are not Arabic (like tags,..)
remove repetitive letters and word which have one letter
apply arabert preprocessing


**Collecting Data **

In [8]:
fields=['Tweet_ID','Tweets_withDecodedemojis','Final_Annotation']
train_dataset = pd.read_csv('SaudiIrony.csv',usecols=fields)
train_Saudi= train_dataset['Tweets_withDecodedemojis']

In [ ]:
train_Saudi

In [9]:
train_d_pos = pd.DataFrame(train_Saudi, columns=[ 'Tweets_withDecodedemojis'])
train_d_pos.rename(columns = {'Tweets_withDecodedemojis':'tweet'}, inplace = True)
train_d_pos['dialect']='SA'
train_d_pos['tweet'] = train_d_pos['tweet'].astype(str)
train_d_pos.head(5)

,tweet,dialect
0,نعم من علامات الجمال تلك الطيبه التي لاترى بل ...,SA
1,المعرفه الجديده والمهارات الجديده واعتماد طرائ...,SA
2,لايشيخ,SA
3,لايشيخ وكورونا بتزيده مناعه يعني كورونا العن ت...,SA
4,لايشيخ و ليه المفروض اشتري بدل,SA


In [10]:
#tunisain
fields=['texts','data_labels']
train_dataset_tuii = pd.read_csv('tun.csv',usecols=fields)
train_dataset_tu=train_dataset_tuii['texts']

In [11]:
#algirian
fields=['id','text']
train_dataset_al = pd.read_csv('datasetAlger.csv',usecols=fields)
train_d_al=train_dataset_al['text']

In [12]:
#egypt
fields=['review']
train_dataset_eg = pd.read_csv('40000-Egyptian-tweets.csv',usecols=fields)
train_d_eg=train_dataset_eg['review']

In [13]:
dd= pd.concat([train_d_al[1:6603], train_d_eg[1:6603]])
dd= pd.concat([dd, train_dataset_tu[1:6603]])
len(dd)

19806

In [14]:
train_d_negative = pd.DataFrame(dd, columns=[ 'tweet'])

In [15]:
train_d_negative['dialect']='NS'

In [16]:
train_d_negative['tweet'] = train_d_negative['tweet'].astype(str)

**Start Pre-processing**

In [17]:
""""the idea of this process is to remove all strange letters from arabic,
and drop duplicate letters in words like "هههههههه" or "لااااااااااا"
"""
def preprocessing_V0_1_0(data):
    for i in tqdm(range(len(data))):
        #get just arabic text
        data['tweet'].iloc[i]=re.sub(r'[u0600-u06FF]+', '', data['tweet'].iloc[i]).strip()
        data['tweet'].iloc[i]=re.sub(r'[a-z]+', '', data['tweet'].iloc[i]).strip()
        #remove duplicate letter
        data['tweet'].iloc[i]=re.sub(r'(.)\1+', r'\1', data['tweet'].iloc[i]).strip()
        #removing symbols
        data['tweet'].iloc[i]=' '.join(w for w in re.split(r"\W", data['tweet'].iloc[i]) if w)
        #remove letters
        data['tweet'].iloc[i]=' '.join(w for w in araby.tokenize(data['tweet'].iloc[i]) if len(w)>1)
    return data

In [ ]:
""""the idea of this process is to check if the word in english or frensh (The most popular foreign languages in the region)
if it is, we drop it,for other words we use aransia to translate it to arabic letters,
and drop duplicate letters in words like "هههههههه" or "لااااااااااا"
"""
#check_frensh= SpellChecker(language='fr')
check_English=SpellChecker()
def check(word):
    if(re.search(r'[a-zA-Z]',word)!=None):
        if word == check_English.correction(word) or word == check_frensh.correction(word):
            return False

    return True

def preprocessing_V0_1_1(data):
    for i in tqdm(range(len(data))):
        #remove links
        data['tweet'].iloc[i] = re.sub(r'http\S+', '',  data['tweet'].iloc[i])
        #remove users nam
        data['tweet'].iloc[i]=' '.join(w for w in re.split(r"@\w*",data['tweet'].iloc[i]) if w)

        #removing symbols
        data['tweet'].iloc[i]=' '.join(w for w in re.split(r"\W", data['tweet'].iloc[i]) if w)


        #remove English word and frensh word
        if re.search(r'[a-zA-Z]',data['tweet'].iloc[i])!=None:
            data['tweet'].iloc[i]=' '.join(w for w in data['tweet'].iloc[i].split() if check(w))
            #use aranisia
            data['tweet'].iloc[i]=transliterate(data['tweet'].iloc[i], source='ma', target='ar' , universal=True)
        #remove duplicate letter
        data['tweet'].iloc[i]=re.sub(r'(.)\1+', r'\1', data['tweet'].iloc[i]).strip()
        #remove letters
        data['tweet'].iloc[i]=' '.join(w for w in araby.tokenize(data['tweet'].iloc[i]) if len(w)>1)
    return data

In [19]:

data__=preprocessing_V0_1_0(train_d_negative.copy())

100%|██████████| 19806/19806 [00:43<00:00, 455.50it/s]


In [ ]:
#data__=preprocessing_V0_1_1(data__.copy())

100%|██████████| 19806/19806 [00:36<00:00, 547.97it/s]


In [20]:
# pre-process the Saudi dialect
data_pos=preprocessing_V0_1_0(train_d_pos.copy())

100%|██████████| 19804/19804 [00:42<00:00, 467.39it/s]


In [ ]:
#data_pos=preprocessing_V0_1_1(data_pos.copy())

100%|██████████| 19804/19804 [00:52<00:00, 379.93it/s]


1- frequent words removing

In [21]:
def freq_words_removal(text, lst_words):
    lst_text = text.split()
    if lst_words is not None:
        lst_text = [word for word in lst_text if word not in lst_words]
    text = " ".join(lst_text)
    return text
wrds = ['مع','لا','على','من','ما','في','الي','هو','انا','أنا','اله']
data_pos["tweet"] = data_pos["tweet"].apply(lambda x: freq_words_removal(x, wrds))

In [22]:
data__["tweet"] = data__["tweet"].apply(lambda x: freq_words_removal(x, wrds))

In [ ]:
len(data_pos)

19804

In [ ]:
len(data__)

19806

Combine the two datasets of Saudi and non-Saudi dialect

In [23]:
data_set=data_pos.append(data__)
len(data_set)

<ipython-input-23-e48a68592b4e>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_set=data_pos.append(data__)


39610

Shuffling

In [ ]:
#Shuffle the dataset
#df = df.reindex(np.random.permutation(df.index))
#df['LABEL'] = 11 # SA is lables as 11
#dataset=dataset[dataset['dialect'].isnull()==False]

In [ ]:
#One-hot encode the lab
#df.loc[df['country'] == 'SA', 'LABEL'] = 0
# I have only one class!!
#from keras.utils.np_utils import to_categorical
#labels = to_categorical(df['LABEL'], num_classes=18)

In [24]:
dataset=data_set.rename(columns={'tweet':'text'})

In [25]:
from arabert.preprocess import ArabertPreprocessor
model_name="bert-base-arabert"
arabert_prep = ArabertPreprocessor(model_name=model_name)

[2023-11-06 12:02:40,687 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [26]:
dataset3=dataset

In [27]:
dataset["text"]=dataset["text"].apply(lambda x:arabert_prep.preprocess(x))

## **create a classification dataset to load the data**

In [36]:
map_label={
    'NS':0,
    'SA':1,
}
label_map={
    0:'NS',
    1:'SA',
}


In [ ]:

# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("lafifi-24/arbert_arabic_dialect_identification")
model = AutoModelForSequenceClassification.from_pretrained("lafifi-24/arbert_arabic_dialect_identification",return_dict=True)

Encoding(num_tokens=68, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
from torch.utils.data import  Dataset

In [37]:
num_labels = 2
max_length = 150

In [38]:

class ClassificationDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(ClassificationDataset).__init__()

      self.text = text
      self.target = target
      self.tokenizer_name = model
      self.tokenizer = tokenizer
      self.max_len = max_len
      self.label_map = label_map


    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())

      inputs = self.tokenizer(
          text,
          max_length=self.max_len,
          padding='max_length',
          truncation=True
        )
      return InputFeatures(**inputs,label= self.target[item])

NameError: ignored

## **Creating datasets**

In [ ]:
train_dataset = ClassificationDataset(
    dataset['text'].to_list(),
    dataset['dialect'].to_list(),
    model,
    max_length,
    map_label
)


In [ ]:
train_dataset

In [ ]:
input = tokenizer(['الكبسه اكله طيبه'],padding=True,truncation=True)#return_tensors='tf')
input

{'input_ids': [[2, 97957, 1034, 43236, 68460, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]}

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='longest', truncation=True)


tokenized_datasets = X_train.map(tokenize_function, batched=True)
#tokenized_datasets= tokenize_function(X_train)

In [ ]:
tokenized_datasets

In [ ]:
#small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(1000))
#small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(19000))
#tokenized_datasets_val= tokenize_function(X_val)

In [ ]:
tokenized_datasets

In [ ]:
X_train.info()

In [ ]:
X_val.info()

In [ ]:
pip install transformers[torch]

In [ ]:
pip install accelerate -U

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
pip install evaluate

In [ ]:
!pip install transformers==4.17

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

### **Kernas**

In [39]:
import numpy as np
sd= dataset
sd['lable']= np.where(sd.dialect=='SA', 1, 0)

In [40]:
dataset2=sd
dataset2['text'] = dataset2['text'].astype(str)
dataset2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39610 entries, 0 to 6602
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     39610 non-null  object
 1   dialect  39610 non-null  object
 2   lable    39610 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [41]:
dataset2.head(2)

,text,dialect,lable
0,نعم علام +ات ال+ جمال تلك ال+ طيبه التي ل+ اتر...,SA,1
1,ال+ معرفه ال+ جديده و+ ال+ مهار +ات ال+ جديده ...,SA,1


In [42]:
X= dataset2.iloc[:,0:2]
y= dataset2['lable']

Split the dataset and compelete the preprossing

In [43]:
#dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25

In [44]:
X_train = X_train.apply(lambda x: str(x[0]))

#X_test = X_test.apply(lambda x:  str(x[0]))

In [45]:
X_val.text=X_val.text.astype(str)#=X_val.apply(lambda x: str(x[0]))

In [46]:
X_val.dialect=X_val.dialect.astype(str)

In [47]:
#tokenize the text
train_encodings = tokenizer(X_train.values.tolist(),
                            truncation=True,
                            padding='longest')#True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
train_encodings

In [49]:
len(train_encodings)

3

In [55]:
X_train.head(2)

text       نعم علام +ات ال+ جمال تلك ال+ طيبه التي ل+ اتر...
dialect                                                   SA
dtype: object

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),y_train))

In [ ]:

#chose the optimizer
optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-5)
#define the loss function
#losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#build the model

# train the model
model.fit(train_dataset.shuffle(len(X_train)).batch(244),
          epochs=3,
          batch_size=244)

In [50]:
import tensorflow as tf

In [52]:
from transformers import AutoTokenizer

tokenized_data = tokenizer(X_train['text'], return_tensors="np", padding=True)
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

labels = y_train#["lable"]  # Label is already an array of 0 and 1

In [ ]:
#data_toknized
#model

new_model = tf.keras.Sequential(model)
new_model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
# Testing
txt = ["فيديوات لي كيطلعو ليك فاش كتوصل لباج 987 فالموقع الازرق"]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
labels = ['SA','QA','KW','AE','OM','JO','PL','BH','LY','EG','SD','IQ','LB','SY','TN','DZ','MA','YE']
print(pred, labels[np.argmax(pred)])

In [ ]:
#model_A = tf.keras.models.load_model("my_model_A")
#model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])
#model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=num_lab

### **Create data Set**

In [61]:

dataset_try= pd.DataFrame(dataset2,columns=[ 'text','lable'])
dataset_try.head()

,text,lable
0,نعم علام +ات ال+ جمال تلك ال+ طيبه التي ل+ اتر...,1
1,ال+ معرفه ال+ جديده و+ ال+ مهار +ات ال+ جديده ...,1
2,لايشيخ,1
3,لايشيخ و+ كورو +نا ب+ تزيد +ه مناع +ه يعني كور...,1
4,لايشيخ لي +ه ال+ مفروض اشترى بدل,1


In [ ]:
pip install datasets

In [65]:
from datasets import Dataset
raw_dataset = Dataset.from_pandas(dataset_try[['text', 'lable']])

# No label preprocessing - this is purely for inference.
def tokenize(batch):
    tokens = tokenizer(batch['text'], truncation = True, padding = True, max_length = 256)
    return tokens

# We lose the progress bar when parallelized :/
tokenized_datasets = raw_dataset.map(tokenize, batched = True, num_proc = 6,
            # Remove any extra columns to avoid a warning when training, not essential though.
                                 remove_columns = raw_dataset.column_names)

Map (num_proc=6):   0%|          | 0/39610 [00:00<?, ? examples/s]

In [66]:
#tokenized_datasets.set_format('torch')
type(tokenized_datasets)

datasets.arrow_dataset.Dataset

# **Devide Dataset for fifi model**

In [29]:
saved_dataset=dataset
dataset_try2= pd.DataFrame(dataset,columns=[ 'text','dialect'])
dataset_try2.head()

,text,dialect
0,نعم علام +ات ال+ جمال تلك ال+ طيبه التي ل+ اتر...,SA
1,ال+ معرفه ال+ جديده و+ ال+ مهار +ات ال+ جديده ...,SA
2,لايشيخ,SA
3,لايشيخ و+ كورو +نا ب+ تزيد +ه مناع +ه يعني كور...,SA
4,لايشيخ لي +ه ال+ مفروض اشترى بدل,SA


In [30]:
from sklearn.model_selection import train_test_split
train, test= train_test_split(dataset_try2, test_size=0.2, random_state=1)

In [53]:
train['dialect'] = train['dialect'].map(map_label)
test['dialect'] = test['dialect'].map(map_label)

In [54]:
test=test[test['dialect'].isnull()==False]
train=train[train['dialect'].isnull()==False]

In [55]:
train['dialect'] = train['dialect'].astype(int)
test['dialect'] = test['dialect'].astype(int)

### **Create Data set as afif**

In [31]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("lafifi-24/arbert_arabic_dialect_identification")
model = AutoModelForSequenceClassification.from_pretrained("lafifi-24/arbert_arabic_dialect_identification",return_dict=True)

In [32]:
from torch.utils.data import  Dataset

In [34]:
num_labels = 2
max_length = 150
max_len=150

In [35]:
map_label={
    'NS':0,
    'SA':1,
}
label_map={
    0:'NS',
    1:'SA',
}

In [49]:
from torch.utils.data import  Dataset
from transformers.data.processors.utils import InputFeatures
class ClassificationDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(ClassificationDataset).__init__()

      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = tokenizer
      self.max_len = max_len
      self.label_map = label_map


    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())

      inputs = self.tokenizer(
          text,
          max_length=self.max_len,
          padding='max_length',
          truncation=True
        )
      return InputFeatures(**inputs,label= self.target[item])

In [56]:
train_dataset = ClassificationDataset(
    train['text'].to_list(),
    train['dialect'].to_list(),
    model,
    max_length,
    map_label
)
test_dataset = ClassificationDataset(
    test['text'].to_list(),
    test['dialect'].to_list(),
    model_name,
    max_len,
    map_label
)

In [117]:
pip install transformers[torch]

In [101]:
pip install accelerate -U

In [38]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [39]:
pip install evaluate

In [ ]:
#!pip install transformers==4.17

In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [41]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")


### **Metrics**

In [42]:

def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'accuracy': acc,
      'recall':macro_recall
  }

### **Training arguments**

In [45]:
training_args = TrainingArguments(
    output_dir= "./train",
    adam_epsilon = 1e-8,
    learning_rate = 2e-5,
    fp16 = False,#True, # enable this when using V100 or T4 GPU
    per_device_train_batch_size = 16, # up to 64 on 16GB with max len of 128
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 4, # use this to scale batch size without needing more memory
    num_train_epochs= 3,
    warmup_ratio = 0,
    do_eval = True,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    save_total_limit=10,
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'eval_loss',
    greater_is_better = False,
    report_to=[]
  )


In [63]:
training_args = TrainingArguments(
    output_dir= "./train",

    #fp16 = False,#True, # enable this when using V100 or T4 GPU
    per_device_train_batch_size = 16, # up to 64 on 16GB with max len of 128
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 4, # use this to scale batch size without needing more memory
    num_train_epochs= 1,
    warmup_ratio = 0
  )

### **Creating the trainer**

In [64]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

### **Tarining**

In [ ]:
trainer.train()

### **Saving the model**

In [ ]:
#you can chose the model from checkpoint
trainer.model.config.label2id = map_label
trainer.model.config.id2label = label_map
trainer.save_model("./model")
train_dataset.tokenizer.save_pretrained("./model")